# Using the Scheduler

## Basic usage

In [ ]:
import numpy as np

from qiskit import IBMQ, QuantumCircuit, transpile, schedule as build_schedule
from qiskit import circuit
from qiskit.pulse import Schedule, Gaussian, SamplePulse, DriveChannel, ControlChannel
from qiskit.test.mock import FakeAlmaden

In [ ]:
circ = QuantumCircuit(1, 1)
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)
circ.measure([0, 1], [0, 1])

circ.draw()

TODO

In [ ]:
backend = FakeAlmaden()

transpiled_circ = transpile(circ, backend)
schedule = build_schedule(transpiled_circ, backend)
schedule.draw()

## Scheduling with custom gate definitions

In [ ]:
custom_q0_h_schedule = Schedule(Gaussian(duration=128, amp=0.045, sigma=8)(DriveChannel(0)))

inst_map = backend.defaults().instruction_schedule_map
inst_map.add('h', 0, custom_q0_h_schedule)

schedule = build_schedule(circ, backend, inst_map=inst_map)

## Building circuits with non-standard gates

Design a new gate by giving it a name, specific qubits, and a pulse `Schedule` definition.

In [ ]:
from qiskit.pulse.schedule import ParameterizedSchedule

random_pulse = lambda x: SamplePulse(np.random.random(max(x, 1)))

def my_perfect_2q_gate_schedule_func(x):
    sched = Schedule()
    sched += random_pulse(x)(DriveChannel(0))
    sched += random_pulse(x)(ControlChannel(0))
    sched += random_pulse(x)(DriveChannel(1))
    return sched

my_perfect_2q_gate_schedule = ParameterizedSchedule(my_perfect_2q_gate_schedule_func, parameters=('x'))

Add definition to instruction map

In [ ]:
inst_map.add('my_perfect_2q_gate', (0, 1), my_perfect_2q_gate_schedule)

Build schedule as usual

In [ ]:
my_perfect_2q_gate = circuit.Gate('my_perfect_2q_gate', 2, [100])

my_perfect_2q_circ = QuantumCircuit(2, 2)
my_perfect_2q_circ.h(0)
my_perfect_2q_circ.append(my_perfect_2q_gate, [0, 1])
my_perfect_2q_circ.h(0)
my_perfect_2q_circ.measure(0, 0)

my_perfect_2q_circ.draw()

Transpile with the new gate

In [ ]:
my_perfect_2q_circ_transpiled = transpile(my_perfect_2q_circ,
                                          coupling_map=[[0, 1]],
                                          basis_gates=backend.configuration().basis_gates + ['my_perfect_2q_gate'])
my_perfect_2q_circ_transpiled.draw()

In [ ]:
my_perfect_2q_pulse_program = build_schedule(my_perfect_2q_circ_transpiled,
                                             backend,
                                             inst_map=inst_map)
my_perfect_2q_pulse_program.draw(plot_range=(0, 2000))